In [1]:
import pandas as pd
import numpy as np
import torch

torch.cuda.empty_cache()

def read_file(fname: str, correct_labels=True) -> pd.DataFrame:
    """Reads a filename, return df with text and labels."""

    df = pd.read_table(fname, sep="\t", header=None, names="text,labels,role".split(","))
    if correct_labels:
        offensive_ids = df.labels != "Acceptable speech"
        df.labels[offensive_ids] = 1
        df.labels[~offensive_ids] = 0

    df = df.drop(columns=["role"])
    return df


def run_hyperparams_optimization(model_name, model_type, lang):
    import torch
    torch.cuda.empty_cache()
    if lang not in {"sl", "hr", "en"}:
        raise AttributeError(f"Language {lang} is not valid")
    eval_file, train_file = f"../data/merged-{lang}.test.tsv" , f"../data/merged-{lang}.train.tsv"
    from simpletransformers.classification import ClassificationModel, ClassificationArgs
    import wandb


    model_args = ClassificationArgs()
    model_args.use_early_stopping = True
    model_args.early_stopping_delta = 0.01
    model_args.early_stopping_metric = "mcc"
    model_args.early_stopping_metric_minimize = False
    model_args.early_stopping_patience = 3
    model_args.evaluate_during_training_steps = 1000

    model_args.evaluate_during_training = True
    model_args.manual_seed = 4
    model_args.use_multiprocessing = True
    model_args.eval_batch_size = 8
    model_args.labels_list = [0, 1]
    model_args.wandb_project = "task3"
    model_args.reprocess_input_data = True
    model_args.overwrite_output_dir = True
    model_args.no_save = True

    sweep_config = {
        "method": "bayes",  # grid, random
        "metric": {"name": "mcc", "goal": "maximize"},
        "parameters": {
            "num_train_epochs":  {"max": 15, "min": 3},
            "learning_rate": {"min": 1e-6, "max": 1e-4},
            "train_batch_size": {"max": 100, "min": 5}
        },
    }

    sweep_id = wandb.sweep(sweep_config, project="task3_"+model_name.replace("/", "_")+model_type)

    train_df = read_file(train_file, correct_labels=True)
    eval_df = read_file(eval_file, correct_labels=True)

    def train():
        # Initialize a new wandb run
        wandb.init()

        # Create a TransformerModel
        model = ClassificationModel(
            model_type,
            model_name,
            use_cuda=True,
            args=model_args,
            sweep_config=wandb.config,
        )
        model.overwrite_output_dir = True
        model.no_save = True
        
        
        # Train the model
        model.train_model(train_df, eval_df=eval_df)

        # Evaluate the model
        model.eval_model(eval_df)

        # Sync wandb
        wandb.join()

    wandb.agent(sweep_id, train, count=50)


In [ ]:
model_name = "xlm-roberta-base"
model_type = "xlmroberta"
language = "en"

run_hyperparams_optimization(model_name, model_type, language)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: ypeq6ds8
Sweep URL: https://wandb.ai/5roop/task3_xlm-roberta-basexlmroberta/sweeps/ypeq6ds8


wandb: Agent Starting Run: mvqg2hej with config:
wandb: 	learning_rate: 7.831626915734913e-05
wandb: 	num_train_epochs: 13
wandb: 	train_batch_size: 79
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 5roop (use `wandb login --relogin` to force relogin)


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias

/home/peterr/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:941: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(


...

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias

Training loss,0.50093
lr,0.0
global_step,1080
_runtime,170
_timestamp,1631637755
_step,27
tp,625
tn,1138
fp,262
fn,277
mcc,0.50731


Training loss,▇▆▇▅▅█▇█▅▇▅▅▄▄▃▃▅▂▃▁▄
lr,▅██▇▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▁▁
global_step,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇▇██
_runtime,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▇▇▇████
_timestamp,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▇▇▇████
_step,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
tp,█▁▅▄
tn,▁███
fp,█▁▁▁
fn,▁█▅▅
mcc,▁▅██


wandb: Agent Starting Run: 8ams3tue with config:
wandb: 	learning_rate: 9.659378022083072e-05
wandb: 	num_train_epochs: 15
wandb: 	train_batch_size: 99
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias

In [ ]:
model_name = "xlm-roberta-large"
model_type = "xlmroberta"
language = "en"

run_hyperparams_optimization(model_name, model_type, language)

In [ ]:
model_name = "distilroberta-base"
model_type = "roberta"
language = "en"

run_hyperparams_optimization(model_name, model_type, language)

In [ ]:
model_name = "roberta-base"
model_type = "roberta"
language = "en"

run_hyperparams_optimization(model_name, model_type, language)